# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### In windows, Using a ODBC DNS (predefined connection name)
##conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')
conn=pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=EYAL-PC;"
                      "Database=COLLEGE;"
                      "Trusted_Connection=yes;")


In [3]:
courses = pd.read_sql("SELECT * FROM Courses",conn)
courses.head()
#courses.dtypes

,CourseId,CourseName,DepartmentID,TeacherId
0,1.0,English Begginers,1.0,15.0
1,2.0,Advanced English,1.0,15.0
2,3.0,Proffesional English,1.0,15.0
3,4.0,Mathematics 101,2.0,2.0
4,5.0,Matematics Advanced,2.0,13.0


In [4]:
len(courses)

31

In [5]:
classrooms = pd.read_sql("SELECT * FROM Classrooms",conn)
classrooms.head()
#classrooms.describe()
#classrooms.dtypes

,CourseId,StudentId,degree
0,10.0,1.0,86.419914
1,10.0,1.0,86.419914
2,10.0,1.0,86.419914
3,10.0,1.0,86.419914
4,11.0,1.0,79.381125


In [6]:
len(classrooms)

3016

In [7]:
departments = pd.read_sql("SELECT * FROM Departments",conn)
departments.head()
#departments.dtypes

,DepartmentId,DepartmentName
0,1.0,English
1,2.0,Science
2,3.0,Arts
3,4.0,Sport


In [8]:
len(departments)

4

In [9]:
classcourse=pd.merge(classrooms, courses, how='left', on=['CourseId'])
classcourse.head()
classcoursesep=pd.merge(classcourse,departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
classcoursesep.head()
#coursedep=pd.merge(courses,departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
#coursedep.head()


,CourseId,StudentId,degree,CourseName,DepartmentID,TeacherId,DepartmentId,DepartmentName
0,10.0,1.0,86.419914,Biology,2.0,3.0,2.0,Science
1,10.0,1.0,86.419914,Biology,2.0,3.0,2.0,Science
2,10.0,1.0,86.419914,Biology,2.0,3.0,2.0,Science
3,10.0,1.0,86.419914,Biology,2.0,3.0,2.0,Science
4,11.0,1.0,79.381125,Genetics,2.0,19.0,2.0,Science


In [10]:
len(classcoursesep)

3016

# Questions
 
 
## Q1. Count the number of students on each department

In [11]:
result1=classcoursesep.groupby('DepartmentName').StudentId.nunique()
print(result1)

DepartmentName
Arts       137
English     84
Science    200
Sport       81
Name: StudentId, dtype: int64


## Q2. How many students have each course of the English department and the total number of students in the department?

In [12]:
classcoursesep1=classcoursesep.copy()
classcoursesep1.head()
result2=classcoursesep1[classcoursesep1.DepartmentName=='English']
result21=result2.groupby('CourseName').StudentId.nunique()
print(result21)


CourseName
Advanced English        24
English Begginers       32
Proffesional English    38
Name: StudentId, dtype: int64


In [13]:
classcoursesep1=classcoursesep.copy()
classcoursesep1.head()
result2=classcoursesep1[classcoursesep1.DepartmentName=='English']
result22=result2.StudentId.nunique()
print(result22)


84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [14]:
classcoursesep3=classcoursesep.copy()
result3=classcoursesep3[classcoursesep3.DepartmentName=='Science']
result31=result3.groupby('CourseName').StudentId.nunique()
df3=pd.DataFrame(result31)
df3['Class_Size_Cat'] = 'Small'
df3.loc[(df3['StudentId']>22),'Class_Size_Cat']='Big'
df3.groupby('Class_Size_Cat').agg({'StudentId':['count']})


,StudentId
,count
Class_Size_Cat,
Big,10
Small,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [15]:
students = pd.read_sql("SELECT * FROM Students",conn)
students.head()
result4=students.groupby('Gender').StudentId.nunique()
print(result4)

Gender
F    165
M    115
Name: StudentId, dtype: int64


## Q5. For which courses the percentage of male/female students is over 70%?

In [26]:
classcoursestu=pd.merge(classcourse,students, how='left', on='StudentId')
classcoursestu['F_CAT'] = 0
classcoursestu.loc[(classcoursestu['Gender']=='F'),'F_CAT']=1
result5=classcoursestu.groupby(['CourseName']).agg({"F_CAT":['count',sum,'mean']})
print(result5)
df5=pd.DataFrame(result5)
df5.head()
df5.loc(['mean']>'0.7')
#df5['Average'] = df5.count / df5.sum
print(df5)

#print (df5.reset_index().sort_values([''mean''],ascending=False).set_index([''mean'']))
#test = df5.sort_values(['mean'], ascending=[False])
#test = df.sort_values(['mean'], ascending=[0])


### יש להביא רק את אלה שעבורם הממוצע גדול מ-0.7- להשלים
#df5[df5.mean>0.70000]
#df5.head()
###להשלים


#result5[result5.F_CAT:('mean']})>"0.7"]
#result51=result5[result5.mean>0.7]
#print(result51)
#print(result5.loc[(result5['mean']>0.7)
#result5.loc[(result5['mean']>0.7)
#classcoursestu.head()
#len(classcoursestu)


SyntaxError: invalid syntax (<ipython-input-26-455fe31eb78e>, line 8)

## Q6. For each department, how many students passed with a grades over 80?

In [22]:
classcoursesep6=classcoursesep.copy()
classcoursesep6.head()
result6=classcoursesep6[classcoursesep6.degree>80]
result61=result6.groupby('DepartmentName').StudentId.nunique()
print(result61)


DepartmentName
Arts        61
English     40
Science    122
Sport       39
Name: StudentId, dtype: int64


## Q7. For each department, how many students passed with a grades under 60?

In [23]:
classcoursesep7=classcoursesep.copy()
classcoursesep7.head()
result7=classcoursesep7[classcoursesep7.degree<60]
result71=result7.groupby('DepartmentName').StudentId.nunique()
print(result71)

DepartmentName
Arts       3
English    2
Science    7
Sport      1
Name: StudentId, dtype: int64


## Q8. Rate the teachers by their average student's grades (in descending order).

In [25]:
teachers = pd.read_sql("SELECT * FROM Teachers",conn)
teachers['Teach_name'] = teachers['FirstName']+" " +teachers['LastName']
teachers.head()
#len(teachers)

classcourse=pd.merge(classrooms, courses, how='left', on=['CourseId'])
classcourse.head()
classcourseteach=pd.merge(classcourse,teachers, how='left', on='TeacherId')
classcourseteach.head()
result8=classcourseteach.groupby(['Teach_name']).agg({'degree':['mean']})
df8=pd.DataFrame(result8)
#df8.columns
#df8.sort_values(by=['mean'])
print(df8)

### לסנן את הטבלה בסדר יורד-להשלים


#result8=classcourseteach.groupby(['CourseName','TeacherId','grade']).StudentId.nunique()
#df8=pd.DataFrame(result8)
#result8.head()



                       degree
                         mean
Teach_name                   
 Alexander Watts    75.668139
 Amanda Dworkin     76.899881
 Caroline Knuesel   75.955453
 Conor Mckittrick   78.699492
 Daniel Bland       78.908281
 Darick Hess        81.432703
 Ernest Stoyanov    78.316150
 Heather Metcalf    80.440242
 Jeffrey Batzel     76.737973
 Jordyn Baron       80.729494
 Kaitlin Delude     76.765296
 Katerina Young     77.406852
 Kaylea Monzingo    79.365537
 Keegan Price       78.814631
 Mackenzie Olsen    78.383348
 Madison Batan      77.743575
 Maxwell Romkee     78.673509
 Mikayla Russell    77.134323
 Robert Aivazian    76.516464
 Tanner Fitzsimons  78.443395
 Terran Beaty       78.495479
 Zachary Healy      79.365867


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [26]:
###    Dep-Course-Teacher-#students
classcourse=pd.merge(classrooms, courses, how='left', on=['CourseId'])
classcourse.head()
classcoursesep=pd.merge(classcourse,departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
classcoursesep.head()
classcoursedepteach=pd.merge(classcoursesep,teachers, how='left', on='TeacherId')
classcoursedepteach.head()
result9=classcoursedepteach.groupby(['DepartmentName','CourseName','Teach_name']).StudentId.nunique()
df9=pd.DataFrame(result9)
print(df9)
df9.head()



                                                        StudentId
DepartmentName CourseName           Teach_name                   
Arts           History of Arts       Caroline Knuesel          32
               Literature            Daniel Bland              39
               Musicology            Madison Batan             30
               Painting              Katerina Young            34
               Sculpture             Robert Aivazian           23
               Theater               Ernest Stoyanov           26
English        Advanced English      Conor Mckittrick          24
               English Begginers     Conor Mckittrick          32
               Proffesional English  Conor Mckittrick          38
Science        Algebra               Alexander Watts           12
               Biology               Jordyn Baron              36
               Chemistry             Jeffrey Batzel            23
               Computer Science      Kaylea Monzingo           38
          

StudentId
DepartmentName CourseName      Teach_name                  
Arts           History of Arts  Caroline Knuesel         32
               Literature       Daniel Bland             39
               Musicology       Madison Batan            30
               Painting         Katerina Young           34
               Sculpture        Robert Aivazian          23

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [27]:
###    Student ID-Student NAME - # COURSE - AVE.grade by Dep.
students = pd.read_sql("SELECT * FROM Students",conn)
students['Stu_name'] = students['FirstName']+" " +students['LastName']
students.head()

classcourse=pd.merge(classrooms, courses, how='left', on=['CourseId'])
classcourse.head()
classcoursesep=pd.merge(classcourse,departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
classcoursesep.head()
classcoursesdepstu=pd.merge(classcoursesep,students, how='left', on='StudentId')
classcoursesdepstu.head()
result10=classcoursesdepstu.groupby(['Stu_name','DepartmentName','CourseName','degree']).StudentId.nunique()
df10=pd.DataFrame(result10)
df10.head()

### להשלים - לייצר ממוצעים + ספירה ברמת מחלקה + מוצע כללי

#result101=df10.groupby(['Stu_name','DepartmentName']).agg({"degree":['mean'],"CourseName":['count']})
#print(result101)

#result10=classcoursesdepstu.groupby(['Stu_name','DepartmentName']).agg({"degree":['mean'],"CourseName":['count']})
#df10=pd.DataFrame(result10)
#df10.head()

#result10=classcoursesdepstu.groupby(['Stu_name','DepartmentName','CourseName','degree']).StudentId.nunique()
#df10=pd.DataFrame(result10)
#df10.head()
#result101=df10.groupby(['Stu_name','DepartmentName']).agg({"degree":['mean'],"CourseName":['count']})
#print(result101)

StudentId
Stu_name       DepartmentName CourseName       degree              
 Aaron Dejoy   Arts           Sculpture        73.889151          1
               Science        Biology          57.978007          1
                              Python           69.148033          1
               Sport          Gymnastics       83.765738          1
 Abbey Singley English        Advanced English 80.199500          1